## Attention Model  (10 pt)

Credit to https://github.com/yandexdataschool/nlp_course/blob/2023/week04_seq2seq/practice_and_homework_pytorch.ipynb

In previous notebook we composed encoder-decoder recurrent neural networks and applied it to the task of machine translation.

![img](https://esciencegroup.files.wordpress.com/2016/03/seq2seq.jpg)
_(img: esciencegroup.files.wordpress.com)_


## Our task today to add additive attention


In [ ]:
#We'll use data from https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench

In [ ]:
#!kaggle datasets download -d devicharith/language-translation-englishfrench

In [ ]:
#!unzip language-translation-englishfrench.zip

In [ ]:

# !pip3 install torch>=1.3.0
!pip3 install subword-nmt &> log
#!wget https://www.dropbox.com/s/yy2zqh34dyhv07i/data.txt?dl=1 -O data.txt
!wget https://raw.githubusercontent.com/yandexdataschool/nlp_course/2020/week04_seq2seq/vocab.py -O vocab.py
# thanks to tilda and deephack teams for the data, Dmitry Emelyanenko for the code :)

In [ ]:
import csv
from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE
tokenizer = WordPunctTokenizer()
def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

# split and tokenize the data
with open('train.en', 'w') as f_src,  open('train.fr', 'w') as f_dst:
  with open('eng_-french.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    for line in csv_reader:
        src_line, dst_line = line[0], line[1]
        f_src.write(tokenize(src_line) + '\n')
        f_dst.write(tokenize(dst_line) + '\n')

# build and apply bpe vocs
bpe = {}
for lang in ['en', 'fr']:
    learn_bpe(open('./train.' + lang), open('bpe_rules.' + lang, 'w'), num_symbols=8000)
    bpe[lang] = BPE(open('./bpe_rules.' + lang))

    with open('train.bpe.' + lang, 'w') as f_out:
        for line in open('train.' + lang):
            f_out.write(bpe[lang].process_line(line.strip()) + '\n')

In [ ]:
bpe['en'].process_line('A quick brown fox jumps over a lazy dog')

### Building vocabularies

We now need to build vocabularies that map strings to token ids and vice versa. We're gonna need these fellas when we feed training data into model or convert output matrices into words.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#data_inp = np.array(open('./train.bpe.ru').read().split('\n'))
data_inp = np.array(open('./train.bpe.fr').read().split('\n'))
data_out = np.array(open('./train.bpe.en').read().split('\n'))

from sklearn.model_selection import train_test_split
train_inp, dev_inp, train_out, dev_out = train_test_split(data_inp, data_out, test_size=3000,
                                                          random_state=42)
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')

In [ ]:
from vocab import Vocab
inp_voc = Vocab.from_lines(train_inp)
out_voc = Vocab.from_lines(train_out)

### Encoder-decoder model

The code below contains a template for a simple encoder-decoder model: single GRU encoder/decoder, no attention or anything. This model is implemented for you as a reference and a baseline for your homework assignment.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

In [ ]:
class BasicModel(nn.Module):
    def __init__(self, inp_voc, out_voc, emb_size=64, hid_size=128):
        """
        A simple encoder-decoder seq2seq model
        """
        super().__init__() # initialize base class to track sub-layers, parameters, etc.

        self.inp_voc, self.out_voc = inp_voc, out_voc
        self.hid_size = hid_size

        self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
        self.emb_out = nn.Embedding(len(out_voc), emb_size)
        self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True)

        self.dec_start = nn.Linear(hid_size, hid_size) #connection between encoder and decoder
        self.dec0 = nn.GRUCell(emb_size, hid_size)
        self.logits = nn.Linear(hid_size, len(out_voc))

    def forward(self, inp, out):
        """ Apply model in training mode """
        initial_state = self.encode(inp)
        return self.decode(initial_state, out)


    def encode(self, inp, **flags):
        """
        Takes symbolic input sequence, computes initial state
        :param inp: matrix of input tokens [batch, time]
        :returns: initial decoder state tensors, one or many
        """
        inp_emb = self.emb_inp(inp)
        batch_size = inp.shape[0]

        enc_seq, [last_state_but_not_really] = self.enc0(inp_emb)
        # enc_seq: [batch, time, hid_size], last_state: [batch, hid_size]

        # note: last_state is not _actually_ last because of padding, let's find the real last_state
        lengths = (inp != self.inp_voc.eos_ix).to(torch.int64).sum(dim=1).clamp_max(inp.shape[1] - 1)
        # print((inp != self.inp_voc.eos_ix).to(torch.int64))
        last_state = enc_seq[torch.arange(len(enc_seq)), lengths]
        # ^-- shape: [batch_size, hid_size]

        dec_start = self.dec_start(last_state)
        return [dec_start] 

    def decode_step(self, prev_state, prev_tokens, **flags):
        """
        Takes previous decoder state and tokens, returns new state and logits for next tokens
        :param prev_state: a list of previous decoder state tensors, same as returned by encode(...)
        :param prev_tokens: previous output tokens, an int vector of [batch_size]
        :return: a list of next decoder state tensors, a tensor of logits [batch, len(out_voc)]
        """
        # prev_gru0_state = prev_state[0]
        [prev_gru0_state, ] = prev_state

        prev_token_embs = self.emb_out(prev_tokens)

        new_gru_activations = self.dec0(prev_token_embs, prev_gru0_state)
        new_dec_state = [new_gru_activations]
        output_logits = self.logits(new_gru_activations)

        return new_dec_state, output_logits

    def decode(self, initial_state, out_tokens, **flags):
        """ Iterate over reference tokens (out_tokens) with decode_step """
        batch_size = out_tokens.shape[0]
        state = initial_state

        # initial logits: always predict BOS
        onehot_bos = F.one_hot(torch.full([batch_size], self.out_voc.bos_ix, dtype=torch.int64),
                               num_classes=len(self.out_voc)).to(device=out_tokens.device)
        first_logits = torch.log(onehot_bos.to(torch.float32) + 1e-9)

        logits_sequence = [first_logits]
        for i in range(out_tokens.shape[1] - 1):
            state, logits = self.decode_step(state, out_tokens[:, i])
            logits_sequence.append(logits)
        return torch.stack(logits_sequence, dim=1)

    def decode_inference(self, initial_state, max_len=100, **flags):
        """ Generate translations from model (greedy version) """
        batch_size, device = len(initial_state[0]), initial_state[0].device
        state = initial_state
        outputs = [torch.full([batch_size], self.out_voc.bos_ix, dtype=torch.int64,
                              device=device)]
        all_states = [initial_state]

        for i in range(max_len):
            state, logits = self.decode_step(state, outputs[-1])
            outputs.append(logits.argmax(dim=-1))
            all_states.append(state)

        return torch.stack(outputs, dim=1), all_states

    def translate_lines(self, inp_lines, **kwargs):
        inp = self.inp_voc.to_matrix(inp_lines).to(device)
        initial_state = self.encode(inp)
        out_ids, states = self.decode_inference(initial_state, **kwargs)
        return self.out_voc.to_lines(out_ids.cpu().numpy()), states


### Training loss

Our training objective is almost the same as it was for neural language models:
$$ L = {\frac1{|D|}} \sum_{X, Y \in D} \sum_{y_t \in Y} - \log p(y_t \mid y_1, \dots, y_{t-1}, X, \theta) $$

where $|D|$ is the __total length of all sequences__, including BOS and first EOS, but excluding PAD.

### Evaluation: BLEU

Machine translation is commonly evaluated with [BLEU](https://en.wikipedia.org/wiki/BLEU) score. This metric simply computes which fraction of predicted n-grams is actually present in the reference translation. It does so for n=1,2,3 and 4 and computes the geometric average with penalty if translation is shorter than reference.

While BLEU [has many drawbacks](http://www.cs.jhu.edu/~ccb/publications/re-evaluating-the-role-of-bleu-in-mt-research.pdf), it still remains the most commonly used metric and one of the simplest to compute.

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(model, inp_lines, out_lines, bpe_sep='@@ ', **flags):
    """
    Estimates corpora-level BLEU score of model's translations given inp and reference out
    Note: if you're serious about reporting your results, use https://pypi.org/project/sacrebleu
    """
    with torch.no_grad():
        translations, _ = model.translate_lines(inp_lines, **flags)
        translations = [line.replace(bpe_sep, '') for line in translations]
        actual = [line.replace(bpe_sep, '') for line in out_lines]
        return corpus_bleu(
            [[ref.split()] for ref in actual],
            [trans.split() for trans in translations],
            smoothing_function=lambda precisions, **kw: [p + 1.0 / p.denominator for p in precisions]
            ) * 100

### Your Attention Required

In this section we want you to improve over the basic model by implementing a simple attention mechanism.

This is gonna be a two-parter: building the __attention layer__ and using it for an __attentive seq2seq model__.

### Attention layer (1 points)

Here you will have to implement a layer that computes a simple additive attention:

Given encoder sequence $ h^e_0, h^e_1, h^e_2, ..., h^e_T$ and a single decoder state $h^d$,

* Compute logits with a 2-layer neural network
$$a_t = linear_{out}(tanh(linear_{e}(h^e_t) + linear_{d}(h_d)))$$
* Get probabilities from logits,
$$ p_t = {{e ^ {a_t}} \over { \sum_\tau e^{a_\tau} }} $$

* Add up encoder states with probabilities to get __attention response__
$$ attn = \sum_t p_t \cdot h^e_t $$

You can learn more about attention layers in the lecture slides or [from this post](https://distill.pub/2016/augmented-rnns/).

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, name, enc_size, dec_size, hid_size, activ=torch.tanh):
        """ A layer that computes additive attention response and weights """
        super().__init__()
        self.name = name
        self.enc_size = enc_size # num units in encoder state
        self.dec_size = dec_size # num units in decoder state
        self.hid_size = hid_size # attention layer hidden units
        self.activ = activ       # attention layer hidden nonlinearity

        # create trainable paramteres like this:
        self.linear_e = nn.Linear(enc_size, hid_size, bias=False)
        self.linear_d = nn.Linear(dec_size, hid_size, bias=False)
        self.linear_out = nn.Linear(hid_size, 1, bias=False)


    def forward(self, enc, dec, inp_mask):
        """
        Computes attention response and weights
        :param enc: encoder activation sequence, float32[batch_size, ninp, enc_size]
        :param dec: single decoder state used as "query", float32[batch_size, dec_size]
        :param inp_mask: mask on enc activatons (0 after first eos), float32 [batch_size, ninp]
        :returns: attn[batch_size, enc_size], probs[batch_size, ninp]
            - attn - attention response vector (weighted sum of enc)
            - probs - attention weights after softmax
        """

        # Compute logits
        logits = self.linear_out(self.activ(self.linear_e(enc) + self.linear_d(dec.unsqueeze(1))))

        # Apply mask - if mask is 0, logits should be -inf or -1e9
        # You may need torch.where
        logits = torch.where(inp_mask == 0, -torch.inf, logits.squeeze(-1))


        # Compute attention probabilities (softmax)
        probs = F.softmax(logits, dim=-1)

        # Compute attention response using enc and probs
        attn = (enc * probs.unsqueeze(-1)).sum(dim=1)

        return attn, probs

### Seq2seq model with attention

You can now use the attention layer to build a network. The simplest way to implement attention is to use it in decoder phase:
![img](https://i.imgur.com/6fKHlHb.png)
_image from distill.pub [article](https://distill.pub/2016/augmented-rnns/)_

On every step, use __previous__ decoder state to obtain attention response. Then feed concat this response to the inputs of next attention layer.

The key implementation detail here is __model state__. Put simply, you can add any tensor into the list of `encode` outputs. You will then have access to them at each `decode` step. This may include:
* Last RNN hidden states (as in basic model)
* The whole sequence of encoder outputs (to attend to) and mask
* Attention probabilities (to visualize)

_There are, of course, alternative ways to wire attention into your network and different kinds of attention. Take a look at [this](https://arxiv.org/abs/1609.08144), [this](https://arxiv.org/abs/1706.03762) and [this](https://arxiv.org/abs/1808.03867) for ideas. And for image captioning/im2latex there's [visual attention](https://arxiv.org/abs/1502.03044)_

In [ ]:
class AttentiveModel(BasicModel):
    def __init__(self, name, inp_voc, out_voc,
                 emb_size=64, hid_size=128, attn_size=128):
        """ Translation model that uses attention. See instructions above. """
        nn.Module.__init__(self)  # initialize base class to track sub-layers, trainable variables, etc.
        self.inp_voc, self.out_voc = inp_voc, out_voc
        self.hid_size = hid_size

        self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
        self.emb_out = nn.Embedding(len(out_voc), emb_size)
        self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True)

        self.attn = AttentionLayer(name, hid_size, hid_size, attn_size)
        self.dec0 = nn.GRUCell(emb_size + attn_size, hid_size)

        self.logits = nn.Linear(hid_size, len(out_voc))
        self.name = name


    def encode(self, inp, **flags):
        """
        Takes symbolic input sequence, computes initial state
        :param inp: matrix of input tokens [batch, time]
        :return: a list of initial decoder state tensors
        """

        # encode input sequence, create initial decoder states
        inp_emb = self.emb_inp(inp)
        batch_size = inp.shape[0]

        enc_seq, _ = self.enc0(inp_emb)

        # apply attention layer from initial decoder hidden state
        mask = self.out_voc.compute_mask(inp)

        dec_start = torch.zeros((batch_size, self.hid_size), device=device)

        first_attn, first_attn_probas = self.attn(enc_seq, dec_start, mask)



        # Build first state: include
        # * initial states for decoder recurrent layers
        # * encoder sequence and encoder attn mask (for attention)
        # * make sure that last state item is attention probabilities tensor

        # first_state = [<...>, first_attn_probas]
        return [dec_start, enc_seq, mask, first_attn, first_attn_probas]

    def decode_step(self, prev_state, prev_tokens, **flags):
        """
        Takes previous decoder state and tokens, returns new state and logits for next tokens
        :param prev_state: a list of previous decoder state tensors
        :param prev_tokens: previous output tokens, an int vector of [batch_size]
        :return: a list of next decoder state tensors, a tensor of logits [batch, n_tokens]
        """

        prev_dec_state, enc_seq, inp_mask, prev_attn, prev_attn_probas = prev_state

        prev_tokens_embs = self.emb_out(prev_tokens)


        new_dec_state = self.dec0(torch.cat([prev_tokens_embs, prev_attn], dim=-1), prev_dec_state)

        output_logits = self.logits(new_dec_state)

        attn, attention_probs = self.attn(enc_seq, new_dec_state, inp_mask)

        return [new_dec_state, enc_seq, inp_mask, attn, attention_probs], output_logits

### Training attentive model

Please reuse the infrastructure you've built for the regular model. I hope you didn't hard-code anything :)

In [ ]:
from IPython.display import clear_output
from tqdm import tqdm, trange

# model = BasicModel( inp_voc, out_voc).to(device)
model = AttentiveModel("attentive", inp_voc, out_voc).to(device)
metrics = {'train_loss': [], 'dev_bleu': []}
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
batch_size = 64

model.train()

In [ ]:
def compute_loss(model, inp, out, **flags):
    """
    Compute loss (float32 scalar) as in the formula above
    :param inp: input tokens matrix, int32[batch, time]
    :param out: reference tokens matrix, int32[batch, time]

    In order to pass the tests, your function should
    * include loss at first EOS but not the subsequent ones
    * divide sum of losses by a sum of input lengths (use voc.compute_mask)
    """
    mask = model.out_voc.compute_mask(out)

    # outputs of the model, [batch_size, out_len, num_tokens]
    logits_seq = model(inp, out) #<YOUR CODE HERE>

    loss = F.cross_entropy(logits_seq.permute(0, 2, 1), out, reduction='none')

    # Note: you can compute loss more efficiently using using F.cross_entropy

    # average cross-entropy over tokens where mask == True
    return (loss*mask).sum() / mask.sum()

In [ ]:
from tqdm import tqdm, trange
for _ in trange(1):
    step = len(metrics['train_loss']) + 1
    batch_ix = np.random.randint(len(train_inp), size=batch_size)
    batch_inp = inp_voc.to_matrix(train_inp[batch_ix]).to(device)
    batch_out = out_voc.to_matrix(train_out[batch_ix]).to(device)



    model.train()
    loss_t = compute_loss(model, batch_inp, batch_out)
    opt.zero_grad()
    loss_t.backward()
    opt.step()
    metrics['train_loss'].append((step, loss_t.item()))

    if step % 100 == 0:
        model.eval()
        dev_bleu = compute_bleu(model, dev_inp, dev_out)
        metrics['dev_bleu'].append(dev_bleu)

        clear_output(True)
        plt.figure(figsize=(12, 4))
        for i, (name, history) in enumerate(sorted(metrics.items())):
            plt.subplot(1, len(metrics), i + 1)
            plt.title(name)
            plt.plot(history)
            plt.grid()
        plt.show()

        print("Mean loss=%.3f, dev BLEU=%.3f" % (np.mean(metrics['train_loss'][-10:]), dev_bleu))

In [ ]:
torch.save(model.state_dict(), 'attention_original.pt')

### Visualizing model attention (1 points)

After training the attentive translation model, you can check it's sanity by visualizing its attention weights.

We provided you with a function that draws attention maps using [`Bokeh`](https://bokeh.pydata.org/en/latest/index.html). Once you managed to produce something better than random noise, please leave them in the notebook or save  bokeh figures and add to your sumbission. You can save bokeh images as screenshots or using this button:

![bokeh_panel](https://github.com/yandexdataschool/nlp_course/raw/2019/resources/bokeh_panel.png)

__Note:__ you're not locked into using bokeh. If you prefer a different visualization method, feel free to use that instead of bokeh.

In [ ]:
import bokeh.plotting as pl
import bokeh.models as bm
from bokeh.io import output_notebook, show
output_notebook()

def draw_attention(inp_line, translation, probs):
    """ An intentionally ambiguous function to visualize attention weights """
    inp_tokens = inp_voc.tokenize(inp_line)
    trans_tokens = out_voc.tokenize(translation)
    probs = probs[:len(trans_tokens), :len(inp_tokens)]

    fig = pl.figure(x_range=(0, len(inp_tokens)), y_range=(0, len(trans_tokens)),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs[::-1]], 0, 0, len(inp_tokens), len(trans_tokens))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens)) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens)) + 0.5, inp_tokens))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens)) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens)) + 0.5, trans_tokens[::-1]))

    show(fig)

In [ ]:
for inp_line, trans_line in zip(dev_inp[::500], model.translate_lines(dev_inp[::500])[0]):
    print(inp_line)
    print(trans_line)
    print()

In [ ]:
for inp_line, trans_line in zip(dev_inp[::500], model.translate_lines(dev_inp[::500])[0]):
    print(inp_line)
    print(trans_line)
    print()

inp = dev_inp[::500]

trans, states = model.translate_lines(inp)

# select attention probs from model state (you may need to change this for your custom model)
# attention_probs below must have shape [batch_size, translation_length, input_length], extracted from states
# e.g. if attention probs are at the end of each state, use np.stack([state[-1] for state in states], axis=1)

probs = [states[i][-1] for i in range(len(states))]
attention_probs = np.stack([state[-1].cpu().detach().numpy() for state in states], axis=1)
for i in range(5):
    draw_attention(inp[i], trans[i], attention_probs[i])

# Does it look fine already? don't forget to save images for anytask!

__Note:__ If the attention maps are not iterpretable, try starting encoder from zeros (instead of dec_start), forcing model to use attention.

## Implement Two different archetectures with attention (8 points)

We want you to find the best model for the task. Use everything you know.

* add attention after RNN or as a hidden state input
* different recurrent units: rnn/gru/lstm; deeper architectures
* bidirectional encoder, different attention methods for decoder (additive, dot-product, multi-head)
* word dropout, training schedules, anything you can imagine
* replace greedy inference with beam search

Describe what you tried and what results you obtained in a short report.

## Attention Model with Attention added after RNN

In [ ]:
def training_loop(model):
    metrics = {'train_loss': [], 'dev_bleu': []}
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    batch_size = 64

    for _ in trange(25000):
        step = len(metrics['train_loss']) + 1
        batch_ix = np.random.randint(len(train_inp), size=batch_size)
        batch_inp = inp_voc.to_matrix(train_inp[batch_ix]).to(device)
        batch_out = out_voc.to_matrix(train_out[batch_ix]).to(device)


        model.train()
        loss_t = compute_loss(model, batch_inp, batch_out)
        opt.zero_grad()
        loss_t.backward()
        opt.step()
        metrics['train_loss'].append((step, loss_t.item()))

        if step % 100 == 0:
            model.eval()
            dev_bleu = compute_bleu(model, dev_inp, dev_out)
            metrics['dev_bleu'].append(dev_bleu)

            clear_output(True)
            plt.figure(figsize=(12, 4))
            for i, (name, history) in enumerate(sorted(metrics.items())):
                plt.subplot(1, len(metrics), i + 1)
                plt.title(name)
                plt.plot(history)
                plt.grid()
            plt.show()

            print("Mean loss=%.3f, dev BLEU=%.3f" % (np.mean(metrics['train_loss'][-10:]), dev_bleu))

def test_model(model):
    for inp_line, trans_line in zip(dev_inp[::500], model.translate_lines(dev_inp[::500])[0]):
        print(inp_line)
        print(trans_line)
        print()

    inp = dev_inp[::500]

    trans, states = model.translate_lines(inp)

    # select attention probs from model state (you may need to change this for your custom model)
    # attention_probs below must have shape [batch_size, translation_length, input_length], extracted from states
    # e.g. if attention probs are at the end of each state, use np.stack([state[-1] for state in states], axis=1)

    probs = [states[i][-1] for i in range(len(states))]
    attention_probs = np.stack([state[-1].cpu().detach().numpy() for state in states], axis=1)
    for i in range(5):
        draw_attention(inp[i], trans[i], attention_probs[i])

In [ ]:
class AttentiveAfterRNNModel(BasicModel):
    def __init__(self, name, inp_voc, out_voc,
                 emb_size=64, hid_size=128, attn_size=128):
        """ Translation model that uses attention. """
        nn.Module.__init__(self)
        self.inp_voc, self.out_voc = inp_voc, out_voc
        self.hid_size = hid_size
        self.emb_size = emb_size
        self.attn_size = attn_size

        self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
        self.emb_out = nn.Embedding(len(out_voc), emb_size)
        self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True)

        # Attention is applied *after* the encoder RNN.
        self.attn = AttentionLayer(name, hid_size, hid_size, attn_size)  # enc_hid_size, dec_hid_size, attn_size
        # Decoder GRUCell takes embedding + attention context vector as input
        self.dec0 = nn.GRUCell(emb_size + hid_size, hid_size)

        self.logits = nn.Linear(hid_size, len(out_voc))
        self.name = name

    def encode(self, inp, **flags):
        """
        Encodes the input sequence and computes the initial decoder state.
        """
        inp_emb = self.emb_inp(inp)
        batch_size = inp.shape[0]

        # enc_seq: [batch, time, hid_size]
        enc_seq, enc_last = self.enc0(inp_emb)

        # Compute the mask for attention.
        mask = self.inp_voc.compute_mask(inp)

        # Initialize the decoder's hidden state.  Often, this is the last encoder state.
        dec_start = enc_last.squeeze(0)  # Remove the layer dimension: [1, batch, hid_size] -> [batch, hid_size]

        # Initial attention context and probabilities (dummy values, will be updated in the first decode step)
        first_attn = torch.zeros((batch_size, self.hid_size), device=device)
        first_attn_probas = torch.zeros((batch_size, inp.shape[1]), device=device) # Example shape, adjust if needed.

        return [dec_start, enc_seq, mask, first_attn, first_attn_probas]


    def decode_step(self, prev_state, prev_tokens, **flags):
        """
        Performs a single decoding step.
        """
        prev_dec_state, enc_seq, mask, _, prev_attn_probas = prev_state

        # Embed the previous output tokens.
        prev_tokens_embs = self.emb_out(prev_tokens)

        # Calculate attention.  Crucially, this happens *before* the decoder RNN.
        attn_context, attn_probas = self.attn(enc_seq, prev_dec_state, mask)

        # Concatenate the embedding and the attention context vector.
        dec_input = torch.cat([prev_tokens_embs, attn_context], dim=-1)

        # Update the decoder's hidden state.
        new_dec_state = self.dec0(dec_input, prev_dec_state)

        # Compute the logits for the next token.
        output_logits = self.logits(new_dec_state)

        # Return the new state and logits.
        return [new_dec_state, enc_seq, mask, attn_context, attn_probas], output_logits

In [ ]:
model = AttentiveAfterRNNModel("AttentiveAfterRNN",inp_voc, out_voc).to(device)

training_loop(model)
torch.save(model.state_dict(), 'Attention after RNN.pt')
test_model(model)

## Attention Model with Beam Search

In [ ]:
class AttentiveModelBeam(AttentiveModel):
    def inference(self, inp, beam_size=5, max_steps=50, eos_token=None):
        """
        Perform beam search inference.
        :param inp: input tokens tensor [batch, time]. (Assumes batch size 1.)
        :param beam_size: number of beams to keep.
        :param max_steps: maximum decoding steps.
        :param eos_token: integer token that marks end-of-sequence. If None, will use self.out_voc.eos.
        :return: a list of token ids representing the best decoded sequence.
        """
        # Get the initial decoder state (assumes batch size 1 for simplicity)
        state = self.encode(inp)
        device = state[0].device

        # Determine start-of-sequence and end-of-sequence tokens.
        start_token = self.out_voc.sos if hasattr(self.out_voc, 'sos') else 0
        eos_token = eos_token if eos_token is not None else (self.out_voc.eos if hasattr(self.out_voc, 'eos') else 1)

        # Each beam is a tuple: (decoded_sequence, state, cumulative_log_prob)
        beams = [([start_token], state, 0.0)]
        finished_beams = []

        for _ in range(max_steps):
            new_beams = []
            for seq, state, score in beams:
                # If EOS is reached, add beam to finished beams
                if seq[-1] == eos_token:
                    finished_beams.append((seq, state, score))
                    continue

                # Decode next token for the current beam.
                prev_token = torch.tensor([seq[-1]], device=device)
                next_state, logits = self.decode_step(state, prev_token)
                log_probs = F.log_softmax(logits, dim=-1)  # [1, vocab_size]

                # Get the top tokens from current beam.
                top_log_probs, top_indices = log_probs.topk(beam_size)

                # Expand each candidate token to form a new beam.
                for i in range(beam_size):
                    token = top_indices[0, i].item()
                    token_log_prob = top_log_probs[0, i].item()
                    new_seq = seq + [token]
                    # Clone the state to avoid in-place modifications in different beams.
                    new_state = self._clone_state(next_state)
                    new_score = score + token_log_prob
                    new_beams.append((new_seq, new_state, new_score))

            if not new_beams:
                break
            # Keep only the top beams (highest cumulative log probability)
            new_beams.sort(key=lambda x: x[2], reverse=True)
            beams = new_beams[:beam_size]

        # Choose best finished beam if any, otherwise the best ongoing beam.
        if finished_beams:
            finished_beams.sort(key=lambda x: x[2], reverse=True)
            best_seq = finished_beams[0][0]
        else:
            beams.sort(key=lambda x: x[2], reverse=True)
            best_seq = beams[0][0]

        return best_seq

    def _clone_state(self, state):
        """
        Clones the list of state tensors.
        :param state: list of tensors representing the decoder state.
        :return: a deep copy of the state list.
        """
        return [s.clone() for s in state]


In [ ]:
model = AttentiveModelBeam("AttentionBeamSearch",inp_voc, out_voc).to(device)

training_loop(model)
torch.save(model.state_dict(), 'AttentionBeamSearch.pt')
test_model(model)

## Attention Model with the attention added after RNN and with Beam Search used for inference

In [ ]:
class AttentiveModelBeamWithAfterRNN(AttentiveAfterRNNModel):
    def inference(self, inp, beam_size=5, max_steps=50, eos_token=None):
        """
        Perform beam search inference.
        :param inp: input tokens tensor [batch, time]. (Assumes batch size 1.)
        :param beam_size: number of beams to keep.
        :param max_steps: maximum decoding steps.
        :param eos_token: integer token that marks end-of-sequence. If None, will use self.out_voc.eos.
        :return: a list of token ids representing the best decoded sequence.
        """
        # Get the initial decoder state (assumes batch size 1 for simplicity)
        state = self.encode(inp)
        device = state[0].device

        # Determine start-of-sequence and end-of-sequence tokens.
        start_token = self.out_voc.sos if hasattr(self.out_voc, 'sos') else 0
        eos_token = eos_token if eos_token is not None else (self.out_voc.eos if hasattr(self.out_voc, 'eos') else 1)

        # Each beam is a tuple: (decoded_sequence, state, cumulative_log_prob)
        beams = [([start_token], state, 0.0)]
        finished_beams = []

        for _ in range(max_steps):
            new_beams = []
            for seq, state, score in beams:
                # If EOS is reached, add beam to finished beams
                if seq[-1] == eos_token:
                    finished_beams.append((seq, state, score))
                    continue

                # Decode next token for the current beam.
                prev_token = torch.tensor([seq[-1]], device=device)
                next_state, logits = self.decode_step(state, prev_token)
                log_probs = F.log_softmax(logits, dim=-1)  # [1, vocab_size]

                # Get the top tokens from current beam.
                top_log_probs, top_indices = log_probs.topk(beam_size)

                # Expand each candidate token to form a new beam.
                for i in range(beam_size):
                    token = top_indices[0, i].item()
                    token_log_prob = top_log_probs[0, i].item()
                    new_seq = seq + [token]
                    # Clone the state to avoid in-place modifications in different beams.
                    new_state = self._clone_state(next_state)
                    new_score = score + token_log_prob
                    new_beams.append((new_seq, new_state, new_score))

            if not new_beams:
                break
            # Keep only the top beams (highest cumulative log probability)
            new_beams.sort(key=lambda x: x[2], reverse=True)
            beams = new_beams[:beam_size]

        # Choose best finished beam if any, otherwise the best ongoing beam.
        if finished_beams:
            finished_beams.sort(key=lambda x: x[2], reverse=True)
            best_seq = finished_beams[0][0]
        else:
            beams.sort(key=lambda x: x[2], reverse=True)
            best_seq = beams[0][0]

        return best_seq

    def _clone_state(self, state):
        """
        Clones the list of state tensors.
        :param state: list of tensors representing the decoder state.
        :return: a deep copy of the state list.
        """
        return [s.clone() for s in state]


In [ ]:
model = AttentiveModelBeamWithAfterRNN("AttentionBeamSearchWithAfterRNN",inp_voc, out_voc).to(device)

training_loop(model)
torch.save(model.state_dict(), 'AttentionBeamSearchWithAfterRNN.pt')
test_model(model)